In [3]:
import pandas as pd
import pyarrow
import sentencepiece
import transformers
print(pyarrow.__version__, transformers.__version__, sentencepiece.__version__)

19.0.1 4.50.3 0.2.0


In [4]:
file_path = f"iwslt2015-en-vi/train-00000-of-00001.parquet"
train = pd.read_parquet(file_path)
print(train.head(20))

                                                   en  \
0   Rachel Pike : The science behind a climate hea...   
1   In 4 minutes , atmospheric chemist Rachel Pike...   
2   I &apos;d like to talk to you today about the ...   
3   Headlines that look like this when they have t...   
4   They are both two branches of the same field o...   
5   Recently the headlines looked like this when t...   
6   That report was written by 620 scientists from...   
7   They wrote almost a thousand pages on the topic .   
8   And all of those pages were reviewed by anothe...   
9   It &apos;s a big community . It &apos;s such a...   
10  Over 15,000 scientists go to San Francisco eve...   
11  And every one of those scientists is in a rese...   
12  For us at Cambridge , it &apos;s as varied as ...   
13  And in each one of these research areas , of w...   
14  And one of the molecules I study is called iso...   
15  The weight of a paper clip is approximately eq...   
16  But despite its very small 